# Qubit rotation

This example shows the basic operation of machine learning framework with a quantum device. A qubit is initilized with a arbitary Rx and Ry rotation and the target state is pure |1> state. After several steps of the iteration. The rotation angle of Rx and Ry will converge to 0 and pi. 

### About this example
The example contains the model compiled with three different configurations of backends and interfaces -- JAX backend, JAX backend with pytorch interface and pytorch backend.
The example also shows how to use state vector propagation mode and tensor network contraction mode. And two methods for obtaining gradient -- back propagation and parameter shift.

# Initialization

In [ ]:
import tedq as qai

# Define the quantum model

### Define the circuit with TeD-Q framework
#### (Remember, if you have multiple measurements, all the measurement results should has the same shape!)

In [ ]:
# Define quantum circuit
def circuitDef(params):
    qai.RX(params[0], qubits=[0])
    qai.RY(params[1], qubits=[0])
    return qai.expval(qai.PauliZ(qubits=[0]))

number_of_qubits = 1
parameter_shapes = [(2,)]

# Quantum circuit construction
circuit = qai.Circuit(circuitDef, number_of_qubits, parameter_shapes = parameter_shapes)

In [ ]:
# visualization of the quantum circuit
drawer = qai.matplotlib_drawer(circuit)
drawer.draw_circuit()

# Circuit compiled with JAX backend
Gradient will obtain from backpropagation by default

### state vector propagation mode

In [ ]:
my_compilecircuit = circuit.compilecircuit(backend="jax")

### tensor network contraction mode

#### Use CoTenGra

In [ ]:
# slicing_opts = {'target_size': 2**28}
# hyper_opt = {'methods':['kahypar'], 'max_time':120, 'max_repeats':12, 'progbar':True, 'minimize':'flops', 'parallel':True, 'slicing_opts':slicing_opts}
# import cotengra as ctg
# my_compilecircuit = circuit.compilecircuit(backend="jax", use_cotengra=ctg, hyper_opt = hyper_opt)

#### Use JDtensorPath (Suggested)
1. 'target_num_slices' is useful if you want to do the contraction in parallel, it will devide the tensor network into pieces and then calculat them in parallel
2. 'math_repeats' means how many times are going to run JDtensorPath to find a best contraction path
3. 'search_parallel' means to run the JDtensorPath in parallel, True means to use all the CPUs, integer number means to use that number of CPUs


In [ ]:
from jdtensorpath import JDOptTN as jdopttn
slicing_opts = {'target_size':2**28, 'repeats':500, 'target_num_slices':None, 'contract_parallel':False}
hyper_opt = {'methods':['kahypar'], 'max_time':120, 'max_repeats':12, 'search_parallel':True, 'slicing_opts':slicing_opts}
my_compilecircuit = circuit.compilecircuit(backend="jax", use_jdopttn=jdopttn, hyper_opt = hyper_opt, tn_simplify = False)

### Define cost function

In [ ]:
def cost(*params):
    return my_compilecircuit(*params)[0]

In [ ]:
new_params = (0.011, 0.012)
cost(*new_params)

### Define optimizer
TeD-Q built-in optimizer

In [ ]:
Optimizer = qai.GradientDescentOptimizer(cost, [0, 1], 0.4, interface="jax")

### Training

In [ ]:
%%time
new_params = (0.011, 0.012)
for i in range(100):
    new_params = Optimizer.step(*new_params)
    if (i + 1) % 5 == 0:
        print("Cost after step {:5d}: {: .7f}".format(i + 1, cost(*new_params)))
        print("Parameters after step {:5d}: {}".format(i + 1, new_params))
print(new_params)
print(cost(*new_params))

### Trained circuit

In [ ]:
trainedCircuit = qai.Circuit(circuitDef, 1, new_params)
drawer = qai.matplotlib_drawer(trainedCircuit)
drawer.full_draw()

# Circuit compiled with JAX backend and pytorch interface
Gradient will obtain from backpropagation by default

### state vector propagation mode

In [ ]:
my_compilecircuit = circuit.compilecircuit(backend="jax", interface="pytorch")

In [ ]:
from jax import numpy as jnp
a = jnp.array([[1,2],[2,4]])

In [ ]:
a.shape

### tensor network contraction mode

#### by using cotengra

In [ ]:
slicing_opts = {'target_size': 2**28}
hyper_opt = {'methods':['kahypar'], 'max_time':120, 'max_repeats':12, 'progbar':True, 'minimize':'flops', 'parallel':True, 'slicing_opts':slicing_opts}
import cotengra as ctg
my_compilecircuit = circuit.compilecircuit(backend="jax", use_cotengra=ctg, hyper_opt = hyper_opt, interface="pytorch")

#### Use JDtensorPath (Suggested)
1. 'target_num_slices' is useful if you want to do the contraction in parallel, it will devide the tensor network into pieces and then calculat them in parallel
2. 'math_repeats' means how many times are going to run JDtensorPath to find a best contraction path
3. 'search_parallel' means to run the JDtensorPath in parallel, True means to use all the CPUs, integer number means to use that number of CPUs


In [ ]:
# from jdtensorpath import JDOptTN as jdopttn
# slicing_opts = {'target_size':2**28, 'repeats':500, 'target_num_slices':None, 'contract_parallel':False}
# hyper_opt = {'methods':['kahypar'], 'max_time':120, 'max_repeats':12, 'search_parallel':True, 'slicing_opts':slicing_opts}
# my_compilecircuit = circuit.compilecircuit(backend="jax", use_jdopttn=jdopttn, hyper_opt = hyper_opt, tn_simplify = False, interface="pytorch")

### Define cost function and optimizer

### Define cost function and optimizer

In [ ]:
def cost(*params):
    return my_compilecircuit(*params)

In [ ]:
# using TeD-Q built-in optimizer
Optimizer = qai.GradientDescentOptimizer(cost, [0, 1], 0.4, interface="pytorch")

### Training

In [ ]:
import torch
a = torch.tensor([0.011], requires_grad= True)
b = torch.tensor([0.012], requires_grad= True)
my_params = (a, b)

In [ ]:
%%time
new_params = my_params
for i in range(100):
    new_params = Optimizer.step(*new_params)
    if (i + 1) % 5 == 0:
        print("Cost after step {:5d}: {}".format(i + 1, cost(*new_params)))
        print("Parameters after step {:5d}: {}".format(i + 1, new_params))
print(new_params)
print(cost(*new_params))

### Trained circuit

In [ ]:
trainedCircuit = qai.Circuit(circuitDef, 1, new_params)
drawer = qai.matplotlib_drawer(trainedCircuit)
drawer.full_draw()

# Circuit compiled with pytorch backend

Gradient will obtain from backpropagation by default

### state vector propagation mode

In [ ]:
# my_compilecircuit = circuit.compilecircuit(backend="pytorch")

### tensor network contraction mode

#### Use CoTenGra

In [ ]:
# slicing_opts = {'target_size': 2**28}
# hyper_opt = {'methods':['kahypar'], 'max_time':120, 'max_repeats':12, 'progbar':True, 'minimize':'flops', 'parallel':True, 'slicing_opts':slicing_opts}
# import cotengra as ctg
# my_compilecircuit = circuit.compilecircuit(backend="pytorch", use_cotengra=ctg, hyper_opt = hyper_opt)

#### Use JDtensorPath (Suggested)
1. 'target_num_slices' is useful if you want to do the contraction in parallel, it will devide the tensor network into pieces and then calculat them in parallel
2. 'math_repeats' means how many times are going to run JDtensorPath to find a best contraction path
3. 'search_parallel' means to run the JDtensorPath in parallel, True means to use all the CPUs, integer number means to use that number of CPUs


In [ ]:
from jdtensorpath import JDOptTN as jdopttn
slicing_opts = {'target_size':2**28, 'repeats':500, 'target_num_slices':None, 'contract_parallel':False}
hyper_opt = {'methods':['kahypar'], 'max_time':120, 'max_repeats':12, 'search_parallel':True, 'slicing_opts':slicing_opts}
my_compilecircuit = circuit.compilecircuit(backend="pytorch", use_jdopttn=jdopttn, hyper_opt = hyper_opt, tn_simplify = False)

### Define cost function and optimizer

In [ ]:
def cost(*params):
    results = my_compilecircuit(*params)
    return results

In [ ]:
# using TeD-Q built-in optimizer
Optimizer = qai.GradientDescentOptimizer(cost, [0, 1], 0.4, interface="pytorch")

### Training

In [ ]:
import torch
a = torch.tensor([0.54], requires_grad= True)
b = torch.tensor([0.12], requires_grad= True)
my_params = (a, b)

In [ ]:
%%time
new_params = my_params
for i in range(100):
    new_params = Optimizer.step(*new_params)
    if (i + 1) % 1 == 0:
        print("Cost after step {:5d}: {}".format(i + 1, cost(*new_params)))
        print("Parameters after step {:5d}: {}".format(i + 1, new_params))
print(new_params)
print(cost(*new_params))

In [ ]:
print("Optimized rotation angles: {}".format(new_params))

In [ ]:
print("Cost: {}".format(cost(*new_params)))

### Using backend's optimizer and training

In [ ]:
from torch import optim
optimizer = optim.Adam([a, b], lr=0.1)
for i in range(500):
    optimizer.zero_grad()
    #print(b.grad)
    loss = cost(*my_params)
    loss.backward()
    optimizer.step()
    if (i + 1) % 5 == 0:
        print("Cost after step {:5d}: {}".format(i + 1, cost(*new_params)))
        print("Parameters after step {:5d}: {}".format(i + 1, new_params))
print(new_params)
print(cost(*new_params))

##  Obtain gradient by parameter shift method

### state vector propagation mode

In [ ]:
# my_compilecircuit = circuit.compilecircuit(backend="pytorch", diff_method = "param_shift")

### tensor network contraction mode

#### Use CoTenGra

In [ ]:
# slicing_opts = {'target_size': 2**28}
# hyper_opt = {'methods':['kahypar'], 'max_time':120, 'max_repeats':12, 'progbar':True, 'minimize':'flops', 'parallel':True, 'slicing_opts':slicing_opts}
# import cotengra as ctg
# my_compilecircuit = circuit.compilecircuit(backend="pytorch", use_cotengra=ctg, hyper_opt = hyper_opt, diff_method = "param_shift")

#### Use JDtensorPath (Suggested)
1. 'target_num_slices' is useful if you want to do the contraction in parallel, it will devide the tensor network into pieces and then calculat them in parallel
2. 'math_repeats' means how many times are going to run JDtensorPath to find a best contraction path
3. 'search_parallel' means to run the JDtensorPath in parallel, True means to use all the CPUs, integer number means to use that number of CPUs


In [ ]:
from jdtensorpath import JDOptTN as jdopttn
slicing_opts = {'target_size':2**28, 'repeats':500, 'target_num_slices':None, 'contract_parallel':False}
hyper_opt = {'methods':['kahypar'], 'max_time':120, 'max_repeats':12, 'search_parallel':True, 'slicing_opts':slicing_opts}
my_compilecircuit = circuit.compilecircuit(backend="pytorch", use_jdopttn=jdopttn, hyper_opt = hyper_opt, tn_simplify = False, diff_method = "param_shift")

### Define cost function and optimizer

In [ ]:
def cost(*params):
    results = my_compilecircuit(*params)
    return results

In [ ]:
# using TeD-Q built-in optimizer
Optimizer = qai.GradientDescentOptimizer(cost, [0, 1], 0.4, interface="pytorch")

### Training

In [ ]:
import torch
a = torch.tensor([0.54], requires_grad= True)
b = torch.tensor([0.12], requires_grad= True)
my_params = (a, b)

In [ ]:
%%time
new_params = my_params
for i in range(100):
    new_params = Optimizer.step(*new_params)
    if (i + 1) % 5 == 0:
        print("Cost after step {:5d}: {}".format(i + 1, cost(*new_params)))
        print("Parameters after step {:5d}: {}".format(i + 1, new_params))
print(new_params)
print(cost(*new_params))

In [ ]:
print("Optimized rotation angles: {}".format(new_params))

In [ ]:
print("Cost: {}".format(cost(*new_params)))

### Using backend's optimizer and training

In [ ]:
from torch import optim
optimizer = optim.Adam([a, b], lr=0.1)
for i in range(500):
    optimizer.zero_grad()
    #print(b.grad)
    loss = cost(*my_params)
    loss.backward()
    optimizer.step()
    if (i + 1) % 5 == 0:
        print("Cost after step {:5d}: {}".format(i + 1, cost(*new_params)))
        print("Parameters after step {:5d}: {}".format(i + 1, new_params))
print(new_params)
print(cost(*new_params))

### Trained circuit

In [ ]:
trainedCircuit = qai.Circuit(circuitDef, 1, new_params)
drawer = qai.matplotlib_drawer(trainedCircuit)
drawer.full_draw()

# Real quantum computer hardware

#### pytorch interface, parameter shift gradient method

#### now only support 1 qubit PauliZ measurement, will be upgrade later

#### check your jobs here: https://quantum-computing.ibm.com/jobs?jobs=circuit

#### caution, this is extremely slow since so many people are queuing for IBMQ free quantum computers

In [ ]:
from qiskit import IBMQ
IBMQ.enable_account('your IBMQ token')

In [ ]:
my_compilecircuit = circuit.compilecircuit(backend="IBMQ_hardware")

### Define cost function and optimizer

In [ ]:
def cost(*params):
    results = my_compilecircuit(*params)
    return results

In [ ]:
# using TeD-Q built-in optimizer
Optimizer = qai.GradientDescentOptimizer(cost, [0, 1], 0.8, interface="pytorch")

### Training

In [ ]:
import torch
a = torch.tensor([0.54], requires_grad= True)
b = torch.tensor([0.12], requires_grad= True)
my_params = (a, b)

In [ ]:
cost(*my_params)

In [ ]:
%%time
new_params = my_params
for i in range(10):
    new_params = Optimizer.step(*new_params)
    if (i + 1) % 2 == 0:
        print("Parameters after step {:5d}: {}".format(i + 1, new_params))
print(new_params)
print(cost(*new_params))